# Finetuning Adam Notebook
------------------
This notebook contains the code used to train the models with the Adam optimizer. 

It functions primarily on Google Colab, but can be adapted to work on local hardware.

It trains only one type of model and searches through hyperparameters with a Grid Search algorithm.

For each model type, minimal adjustments need to be made before running the code again.

In [ ]:
# This script mounts Google Drive in a Google Colab environment.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Set the current working directory to a specific path in Google Drive.
%cd /content/drive/MyDrive/OptiML/repo/OptML-project

/content/drive/MyDrive/OptiML/repo/OptML-project


## Global Imports

In [ ]:
# Import necessary libraries for the project.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from Functions.implementations import * # Import custom implementations, these contain the model definitions and training functions.

# Training with Adam optimizer :

In [ ]:
import os

#only for google Colab :

# Set a path in your Google Drive
csv_path = "/content/drive/MyDrive/OptiML/Results/Adam_VGG_new.csv"
save_path = "/content/drive/MyDrive/OptiML/Results/Adam_VGG_new"
# For local machine, set the path to your desired location

# Set the number of epochs and evaluation interval
epochs = 100
eval_interval = 10 # Interval for evaluation and saving results

# Create the CSV with headers if it doesn't exist
if not os.path.exists(csv_path):
    columns = ["learning_rate", "beta_1","beta_2"] +  [f"epoch_{i}" for i in range(epochs//eval_interval, epochs + 1, eval_interval)] + ["Test"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

set_seed(42) # Set seed for reproducibility through custom function as done throughtout the project

In [ ]:
from itertools import product

# Define the device to use for training (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to get data loaders for training, validation, and testing
train_loader,valid_loader, test_loader = get_data_loaders(batch_size=128)

# Set up a dictionary to store results
results_grid = {}

# Define the hyperparameters for the grid search (for VGG-like model, extra values wer used to find the best range to tune in)
learning_rates = [1e-3, 5e-4, 1e-4] # Extra parameters for VGG-like model : 1e-2, 5e-3, 2e-3
beta_1s = [0.95, 0.9, 0.8] # Extra parameters for VGG-like model : 0.5
beta_2s = [0.99, 0.999, 0.9999] # Extra parameters for VGG-like model : 0.98 and 0.5

# Iterate over all combinations of lr , beta_1, and beta_2
for lr, beta_1, beta_2 in product(learning_rates, beta_1s, beta_2s):
    scores, model = train_and_return_evaluation_Adam(
        VGGLike, # Model type that needs to be trained, needs to be changed for different model types. Options are defined in the implementations.py file.
        lr=lr,
        beta_1=beta_1,
        beta_2=beta_2,
        train_loader=train_loader,
        valid_loader=valid_loader,
        test_loader=test_loader,
        device=device,
        epochs=epochs,
        eval_interval=eval_interval
    )
    results_grid[(lr, beta_1, beta_2)] = scores # Store the scores for each combination of hyperparameters

    # Extract F1 scores only, for CSV export
    f1_scores = [f1 for (_, _, _, f1) in scores]
    
    # Create a row for the CSV
    row = [lr, beta_1, beta_2] + f1_scores

    # Append to CSV
    df_row = pd.DataFrame([row])
    df_row.to_csv(csv_path, mode='a', header=False, index=False)

    # Save the model
    torch.save(model.state_dict(), save_path + f"/VGG_lr_{lr}_beta1_{beta_1}_beta2_{beta_2}.pth") # Save the model with a unique name based on hyperparameters and model type